In [1]:
import numpy as np
import pandas as pd
import vartools as vt

## Commodities

In [2]:
data = pd.read_excel('19.- Data_Examen_2.xlsx', sheet_name=1)
data = data.set_index('Date')
# Sugar, Orange
data.sort_index(inplace=True)
data

,Bid,Ask,Bid.1,Ask.1
Date,,,,
2021-12-15,0.1924,0.1925,1.3345,1.3480
2021-12-16,0.1937,0.1938,1.3725,1.3850
2021-12-17,0.1912,0.1913,1.3855,1.3890
2021-12-20,0.1859,0.1860,1.4180,1.4225
2021-12-21,0.1873,0.1875,1.4070,1.4200
...,...,...,...,...
2024-12-06,0.2173,0.2175,5.0005,5.0125
2024-12-09,0.2148,0.2149,4.9305,4.9565
2024-12-10,0.2106,0.2107,4.9970,5.0090


In [3]:
posiciones = [30_000*15_000, 250*112_000]
conf = 0.95
long = False

In [4]:
vt.var_apl(data, posiciones, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.0273,"6,392,309.4393"
1,VaR Ajustado Promedio,0.0311,"7,280,695.6981"
2,VaR Ajustado Estresado,0.0381,"8,938,001.4074"
3,C-VaR,0.0408,"9,557,222.1835"
4,C-VaR Ajustado Promedio,0.0446,"10,445,608.4423"
5,C-VaR Ajustado Estresado,0.0516,"12,102,914.1516"


In [5]:
def var_apl(data: pd.DataFrame, posiciones: list | np.ndarray, conf: float, long: bool):
    data = data.sort_index()

    # Bid y Ask
    bid_columns = [col for col in data.columns if 'Bid' in col] # Selecciona las columnas que contienen 'Bid'
    ask_columns = [col for col in data.columns if 'Ask' in col] # Selecciona las columnas que contienen 'Ask'

    # Mid
    mid_columns = [f'Mid.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Mid
    data[mid_columns] = (data[bid_columns].values + data[ask_columns].values) / 2

    # Spreads
    spread_columns = [f'Spread.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Spread
    data[spread_columns] = (data[ask_columns].values - data[bid_columns].values) / data[mid_columns].values

    # Returns
    return_columns = [f'Return.{i}' for i in range(len(mid_columns))] # Se crea una lista con los nombres de las columnas de Return
    data[return_columns] = data[mid_columns].pct_change()

    # Weights
    value = posiciones * data[mid_columns].iloc[-1].values
    pv = np.sum(value)
    w = value / pv

    # Portfolio return
    data['port_ret'] = np.dot(data[return_columns], w)

    # VaR calculation
    var_pct = np.percentile(data['port_ret'].dropna(), 100 - conf*100) if long else np.percentile(data['port_ret'].dropna(), conf*100)
    var_cash = pv * var_pct

    # C-VaR calculation
    cvar_pct = data['port_ret'][data['port_ret'] < var_pct].dropna().mean() if long else data['port_ret'][data['port_ret'] > var_pct].dropna().mean()
    cvar_cash = pv * cvar_pct

    # Liquidity cost
    cl_prom = data[spread_columns].mean()
    cl_estr = np.percentile(data[spread_columns], 99, axis=0)

    # VaR adjusted by liquidity cost

    var_apl_prom, var_apl_estr = np.abs(((var_pct - np.dot(w, cl_prom), var_pct - np.dot(w, cl_estr)) if long 
                                else (var_pct + np.dot(w, cl_prom), var_pct + np.dot(w, cl_estr))))

    var_apl_prom_cash, var_apl_estr_cash = np.abs(((var_cash - np.dot(value, cl_prom), var_cash - np.dot(value, cl_estr)) if long 
                                            else (var_cash + np.dot(value, cl_prom), var_cash + np.dot(value, cl_estr))))
    
    # C-VaR adjusted by liquidity cost

    cvar_apl_prom, cvar_apl_estr = np.abs(((cvar_pct - np.dot(w, cl_prom), cvar_pct - np.dot(w, cl_estr)) if long
                                    else (cvar_pct + np.dot(w, cl_prom), cvar_pct + np.dot(w, cl_estr))))
    
    cvar_apl_prom_cash, cvar_apl_estr_cash = np.abs(((cvar_cash - np.dot(value, cl_prom), cvar_cash - np.dot(value, cl_estr)) if long
                                            else (cvar_cash + np.dot(value, cl_prom), cvar_cash + np.dot(value, cl_estr))))

    resultados = pd.DataFrame({
        'Métrica': ['VaR', 'VaR Ajustado Promedio', 'VaR Ajustado Estresado', 'C-VaR', 'C-VaR Ajustado Promedio', 'C-VaR Ajustado Estresado'],
        'Porcentaje': [np.abs(var_pct), var_apl_prom, var_apl_estr, np.abs(cvar_pct), cvar_apl_prom, cvar_apl_estr],
        'Cash': [np.abs(var_cash), var_apl_prom_cash, var_apl_estr_cash, np.abs(cvar_cash), cvar_apl_prom_cash, cvar_apl_estr_cash]
    })

    return np.dot(value, cl_prom)

In [6]:
var_apl(data, posiciones, conf, long)

888386.2588055194

## Bonds

In [7]:
data = pd.read_excel('19.- Data_Examen_2.xlsx', sheet_name=3)
# Estados Unidos, Europa, Canadá
data = data.set_index('Date')
data = data.sort_index()
data

,Bid,Ask,Volume,Bid.1,Ask.1,Volume.1,Bid.2,Ask.2,Volume.2
Date,,,,,,,,,
2019-12-19,128.1719,128.1875,"1,205,409.0000",100.4050,100.4100,"19,631.0000",138.1600,138.1900,"81,465.0000"
2019-12-20,128.2031,128.2344,"806,749.0000",100.4050,100.4100,"23,520.0000",138.2000,138.2400,"79,995.0000"
2019-12-23,128.1094,128.1250,"620,132.0000",100.4050,100.4100,"24,777.0000",137.4800,137.4900,"90,434.0000"
2019-12-24,128.3594,128.3750,"398,672.0000",100.4100,100.4150,"120,733.0000",137.8900,137.9000,"93,855.0000"
2019-12-26,128.4219,128.4375,"402,206.0000",100.4150,100.4200,"35,608.0000",138.3700,138.3800,"74,247.0000"
...,...,...,...,...,...,...,...,...,...
2024-12-09,111.1250,111.1406,"1,148,645.0000",97.1750,97.1800,"114,236.0000",124.2700,124.2800,"143,099.0000"
2024-12-10,110.9531,110.9688,"1,241,344.0000",97.1650,97.1700,"153,413.0000",124.4300,124.4500,"173,419.0000"
2024-12-11,110.6719,110.6875,"1,650,365.0000",97.1600,97.1650,"163,973.0000",123.7300,123.7600,"182,071.0000"


In [8]:
posiciones = [12_765, 10_976, 11_764]
conf = 0.99
long = True

In [9]:
vt.var_apl(data, posiciones, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.0081,"31,523.2535"
1,VaR Ajustado Promedio,0.0083,"32,585.1177"
2,VaR Ajustado Estresado,0.0110,"43,226.2197"
3,C-VaR,0.0100,"39,171.4385"
4,C-VaR Ajustado Promedio,0.0103,"40,233.3027"
5,C-VaR Ajustado Estresado,0.0130,"50,874.4048"


In [10]:
volumes = data[['Volume', 'Volume.1', 'Volume.2']]
volumes = volumes.sort_index()
volumes

,Volume,Volume.1,Volume.2
Date,,,
2019-12-19,"1,205,409.0000","19,631.0000","81,465.0000"
2019-12-20,"806,749.0000","23,520.0000","79,995.0000"
2019-12-23,"620,132.0000","24,777.0000","90,434.0000"
2019-12-24,"398,672.0000","120,733.0000","93,855.0000"
2019-12-26,"402,206.0000","35,608.0000","74,247.0000"
...,...,...,...
2024-12-09,"1,148,645.0000","114,236.0000","143,099.0000"
2024-12-10,"1,241,344.0000","153,413.0000","173,419.0000"
2024-12-11,"1,650,365.0000","163,973.0000","182,071.0000"


In [11]:
n_shares = [12_765, 10_976, 11_764]

In [12]:
adv = volumes.iloc[-90:].mean(axis=0)
adv

Volume     2,166,662.6667
Volume.1     233,054.0889
Volume.2     158,502.0000
dtype: float64

In [13]:
(n_shares/adv)

Volume     0.0059
Volume.1   0.0471
Volume.2   0.0742
dtype: float64

In [14]:
def var_apl(data: pd.DataFrame, posiciones: list | np.ndarray, conf: float, long: bool):
    data = data.sort_index()

    # Bid y Ask
    bid_columns = [col for col in data.columns if 'Bid' in col] # Selecciona las columnas que contienen 'Bid'
    ask_columns = [col for col in data.columns if 'Ask' in col] # Selecciona las columnas que contienen 'Ask'

    # Mid
    mid_columns = [f'Mid.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Mid
    data[mid_columns] = (data[bid_columns].values + data[ask_columns].values) / 2

    # Spreads
    spread_columns = [f'Spread.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Spread
    data[spread_columns] = (data[ask_columns].values - data[bid_columns].values) / data[mid_columns].values

    # Returns
    return_columns = [f'Return.{i}' for i in range(len(mid_columns))] # Se crea una lista con los nombres de las columnas de Return
    data[return_columns] = data[mid_columns].pct_change()

    # Weights
    value = posiciones * data[mid_columns].iloc[-1].values
    pv = np.sum(value)
    w = value / pv

    # Portfolio return
    data['port_ret'] = np.dot(data[return_columns], w)

    # VaR calculation
    var_pct = np.percentile(data['port_ret'].dropna(), 100 - conf*100) if long else np.percentile(data['port_ret'].dropna(), conf*100)
    var_cash = pv * var_pct

    # C-VaR calculation
    cvar_pct = data['port_ret'][data['port_ret'] < var_pct].dropna().mean() if long else data['port_ret'][data['port_ret'] > var_pct].dropna().mean()
    cvar_cash = pv * cvar_pct

    # Liquidity cost
    cl_prom = data[spread_columns].mean()
    cl_estr = np.percentile(data[spread_columns], 99, axis=0)

    # VaR adjusted by liquidity cost

    var_apl_prom, var_apl_estr = np.abs(((var_pct - np.dot(w, cl_prom), var_pct - np.dot(w, cl_estr)) if long 
                                else (var_pct + np.dot(w, cl_prom), var_pct + np.dot(w, cl_estr))))

    var_apl_prom_cash, var_apl_estr_cash = np.abs(((var_cash - np.dot(value, cl_prom), var_cash - np.dot(value, cl_estr)) if long 
                                            else (var_cash + np.dot(value, cl_prom), var_cash + np.dot(value, cl_estr))))
    
    # C-VaR adjusted by liquidity cost

    cvar_apl_prom, cvar_apl_estr = np.abs(((cvar_pct - np.dot(w, cl_prom), cvar_pct - np.dot(w, cl_estr)) if long
                                    else (cvar_pct + np.dot(w, cl_prom), cvar_pct + np.dot(w, cl_estr))))
    
    cvar_apl_prom_cash, cvar_apl_estr_cash = np.abs(((cvar_cash - np.dot(value, cl_prom), cvar_cash - np.dot(value, cl_estr)) if long
                                            else (cvar_cash + np.dot(value, cl_prom), cvar_cash + np.dot(value, cl_estr))))

    resultados = pd.DataFrame({
        'Métrica': ['VaR', 'VaR Ajustado Promedio', 'VaR Ajustado Estresado', 'C-VaR', 'C-VaR Ajustado Promedio', 'C-VaR Ajustado Estresado'],
        'Porcentaje': [np.abs(var_pct), var_apl_prom, var_apl_estr, np.abs(cvar_pct), cvar_apl_prom, cvar_apl_estr],
        'Cash': [np.abs(var_cash), var_apl_prom_cash, var_apl_estr_cash, np.abs(cvar_cash), cvar_apl_prom_cash, cvar_apl_estr_cash]
    })

    return cl_prom, cl_estr

In [15]:
var_apl(data, posiciones, conf, long)

(Spread.0   0.0002
 Spread.1   0.0001
 Spread.2   0.0005
 dtype: float64,
 array([0.00036532, 0.00010396, 0.00766038]))